In [1]:
import tensorflow as tf
from tensorflow import keras

C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\BastOS\anaconda3\envs\timeseries\lib\site-pack

In [2]:
from numpy import array

# split a univariate sequence into sample
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

# define univariate time series
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
print(series.shape)
# transform to a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)

# show each sample
for i in range(len(X)):
    print(X[i], y[i])

    
# transform input from [samples, features] to [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)
    


(10,)
(7, 3) (7,)
[1 2 3] 4
[2 3 4] 5
[3 4 5] 6
[4 5 6] 7
[5 6 7] 8
[6 7 8] 9
[7 8 9] 10
(7, 3, 1)


In [3]:
# example of defining a dataset
from numpy import array
# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
    
data = array(data)
print(data[:5, :])
print(data.shape)

# drop time
data = data[:, 1]
print(data.shape)

samples = list()
lenght = 200

for i in range(0, n, lenght):
    sample = data[i:i+lenght]
    samples.append(sample)
    
print(len(samples))

# 25 samples 200 timesteps per sample et 1 feature

# convert to array numpy
samples = array(samples)
print(samples.shape)

# convert to 3D array 
data = samples.reshape(samples.shape[0], samples.shape[1], 1)
print(data.shape)

[[ 1 10]
 [ 2 20]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
(5000, 2)
(5000,)
25
(25, 200)
(25, 200, 1)


In [4]:
import pandas as pd

series = pd.read_csv("E:/datasets/daily_min_temperatures/daily-min-temperatures.csv", header=0, index_col=0, parse_dates=True,
squeeze=True)
print(type(series))
print(series.head())

<class 'pandas.core.series.Series'>
Date
1981-01-01    20.7
1981-01-02    17.9
1981-01-03    18.8
1981-01-04    14.6
1981-01-05    15.8
Name: Temp, dtype: float64


In [5]:
lenght_series = len(series)
lenght_sequence = 365

samples = list()

for i in range(0, lenght_series, lenght_sequence):
    sample = series[i:i+lenght_sequence]
    samples.append(sample)

samples = array(samples)
print(samples.shape)
df = samples.reshape(samples.shape[0], samples.shape[1], 1)
print(df.shape)

(10, 365)
(10, 365, 1)


# MLP Model

## Univariate timeseries

In [6]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [7]:
# define model
model = keras.models.Sequential([
    keras.layers.Dense(100, activation='relu', input_dim=n_steps),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [9]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))

yhat = model.predict(x_input, verbose=0)
yhat

array([[101.2443]], dtype=float32)

## Multivariate timeseries

Two main models that we may require with multivariate time series data:
- Multiple Input Series
- Multiple Parallel Series

### Multiple Input Series

In [10]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [11]:
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape(len(in_seq1), 1)
in_seq2 = in_seq2.reshape(len(in_seq2), 1)
out_seq = out_seq.reshape(len(out_seq), 1)

In [12]:
from numpy import hstack

dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [13]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)


In [14]:
X, y = split_sequences(dataset, 3)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [15]:
y

array([ 65,  85, 105, 125, 145, 165, 185])

In [16]:
print(X[0])

[[10 15]
 [20 25]
 [30 35]]


In [17]:
X.shape

(7, 3, 2)

In [18]:
# We need to flatten the input in order to fit MLP
# [10, 15, 20, 25, 30, 35]

# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape(X.shape[0], n_input)

In [19]:
X.shape

(7, 6)

In [20]:
X[0]

array([10, 15, 20, 25, 30, 35])

In [21]:
# define the model

model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_dim=n_input),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [22]:
model.fit(X, y, epochs=2000, verbose=0)

In [23]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
print(x_input.shape)
x_input = x_input.reshape(1, n_input)

(3, 2)


In [24]:
x_input

array([[ 80,  85,  90,  95, 100, 105]])

In [25]:
yhat = model.predict(x_input)
yhat

array([[205.34091]], dtype=float32)

#### Multi-headed MLP Model

In [26]:
n_steps = 3
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [27]:
# first input model
visible1 = keras.Input(shape=(n_steps,))
dense1 = keras.layers.Dense(100, activation="relu")(visible1)

# second input model
visible2 = keras.Input(shape=(n_steps,))
dense2 = keras.layers.Dense(100, activation="relu")(visible2)

# merge input models
merge = keras.layers.concatenate([dense1, dense2])
output = keras.layers.Dense(1)(merge)

# connect input and output models
model = keras.models.Model(inputs=[visible1, visible2], outputs=output)

# compile the model
model.compile(optimizer="adam", loss="mse")

In [28]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 100)          400         input_1[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 100)          400         input_2[0][0]                    
______________________________________________________________________________________________

In [29]:
# separate input data
X1 = X[:, :, 0]
X2 = X[:, :, 1]

In [30]:
# fit model
model.fit([X1, X2], y, epochs=2000, verbose=0)

In [31]:
# predict data
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape(1, n_steps)
x2 = x_input[:, 1].reshape(1, n_steps)

yhat = model.predict([x1, x2])
yhat

array([[205.75322]], dtype=float32)

### Multiple Parallel Series

In [32]:
# split multivariate sequence into samples (forecast each ts)
def split_sequences_multiforecast(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps

        if end_ix > len(sequences) - 1:
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [33]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences_multiforecast(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [34]:
# flatten input 
n_input = X.shape[1] * X.shape[2]
X = X.reshape(X.shape[0], n_input)

In [35]:
# determine the number of outputs
n_output = y.shape[1]

In [36]:
# define model
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_dim=n_input),
    keras.layers.Dense(n_output)
])

model.compile(optimizer="adam", loss="mse")

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               1000      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 303       
Total params: 1,303
Trainable params: 1,303
Non-trainable params: 0
_________________________________________________________________


In [37]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [38]:
# shape of single prediction [1, 3, 3]

# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape(1, n_input)
print(x_input.shape)
print(x_input)

yhat = model.predict(x_input)
print(yhat)

(1, 9)
[[ 70  75 145  80  85 165  90  95 185]]
[[100.622665 106.04698  206.12431 ]]


### Multi-output MLP Model

In [60]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences_multiforecast(dataset, n_steps)
print(X.shape, y.shape)

(6, 3, 3) (6, 3)


In [61]:
# define model
visible = keras.Input(shape=(n_input,))
dense = keras.layers.Dense(100, activation="relu")(visible)

In [62]:
# define output 1
output1 = keras.layers.Dense(1)(dense)

# define output 2
output2 = keras.layers.Dense(1)(dense)

# define output 3
output3 = keras.layers.Dense(1)(dense)

In [63]:
# tie together
model = keras.models.Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer="adam", loss="mse")

In [64]:
# flatten input 
n_input = X.shape[1] * X.shape[2]
X = X.reshape(X.shape[0], n_input)

In [65]:
y.shape

(6, 3)

In [66]:
y

array([[ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [67]:
# separate output
y1 = y[:, 0].reshape(y.shape[0], 1)
y2 = y[:, 1].reshape(y.shape[0], 1)
y3 = y[:, 2].reshape(y.shape[0], 1)

In [68]:
# fit model
model.fit(X, [y1,y2,y3], epochs=2000, verbose=0)

In [76]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape(1, n_input)
yhat = model.predict(x_input, verbose=0)
print(yhat)

[array([[99.991585]], dtype=float32), array([[104.98602]], dtype=float32), array([[205.2653]], dtype=float32)]


## Multi-step MLP Models

In [96]:
# split a univariate sequence into samples (with more forecast output)
def split_sequence_unimultiforecast(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out
        if out_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix], sequences[end_ix:out_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [97]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence_unimultiforecast(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


In [98]:
# define model
model = keras.models.Sequential([
    keras.layers.Dense(100, activation='relu', input_dim=n_steps_in),
    keras.layers.Dense(n_steps_out)
])

model.compile(optimizer='adam', loss='mse')

In [101]:
X.shape

(5, 3)

In [102]:
model.fit(X, y, epochs=2000, verbose=0)

In [104]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape(1, n_steps_in)
yhat = model.predict(x_input)
print(yhat)

[[100.881096 110.5861  ]]


## Multiple Input Multi-step Output

In [112]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out-1
        if out_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [113]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [116]:
# flatten the input
n_input = X.shape[1] * X.shape[2]
X = X.reshape(X.shape[0], n_input)

In [117]:
# define model
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_dim=n_input),
    keras.layers.Dense(n_steps_out)
])

model.compile(optimizer="adam", loss="mse")

In [118]:
# fit the model
model.fit(X, y, epochs=2000, verbose=0)

In [121]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape(1, n_input)
yhat = model.predict(x_input)
print(yhat)

[[185.31714 205.02153]]


### Multiple Parallel Input and Multi-step Output

In [ ]:
10, 15, 25
20, 25, 45
30, 35, 65

40, 45, 85
50, 55, 105

In [128]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_ix = end_ix + n_steps_out
        if out_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [141]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [142]:
y[0]

array([[ 40,  45,  85],
       [ 50,  55, 105]])

In [143]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X = X.reshape(X.shape[0], n_input)

n_output = y.shape[1] * y.shape[2]
y = y.reshape(y.shape[0], n_output)

In [144]:
# define mode
model = keras.models.Sequential([
    keras.layers.Dense(100, activation="relu", input_dim=n_input),
    keras.layers.Dense(n_output)
])

model.compile(optimizer="adam", loss="mse")

In [145]:
model.fit(X, y, epochs=2000, verbose=0)

In [148]:
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape(1, n_input)
yhat = model.predict(x_input)
print(yhat)

[[ 90.55892  95.28827 185.12033 100.75181 104.77937 204.87148]]
